In [7]:
import pandas as pd
import requests, zipfile, io

response = requests.get('https://apps.fas.usda.gov/psdonline/downloads/psd_coffee_csv.zip')
zf = zipfile.ZipFile(io.BytesIO(response.content))
usda = pd.read_csv(zf.open('psd_coffee.csv'))

In [17]:
usda.head()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,711100,"Coffee, Green",AG,Algeria,2003,2012,6,29,Arabica Production,2,(1000 60 KG BAGS),0.0
1,711100,"Coffee, Green",AG,Algeria,2003,2012,6,90,Bean Exports,2,(1000 60 KG BAGS),0.0
2,711100,"Coffee, Green",AG,Algeria,2003,2012,6,58,Bean Imports,2,(1000 60 KG BAGS),1800.0
3,711100,"Coffee, Green",AG,Algeria,2003,2012,6,20,Beginning Stocks,2,(1000 60 KG BAGS),0.0
4,711100,"Coffee, Green",AG,Algeria,2003,2012,6,125,Domestic Consumption,2,(1000 60 KG BAGS),1805.0


In [33]:
def country_query(country, year_type='Market_Year', attribute=''):
    df = usda[usda['Country_Name'] == country]
    if attribute:
        df = df[df['Attribute_Description'] == attribute]
    return df.pivot(
        index='Attribute_Description', 
        columns=year_type, 
        values='Value'
    )

country_query('Guatemala', attribute='Arabica Production')

Market_Year,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Attribute_Description,,,,,,,,,,,,,,,,,,,,,
Arabica Production,1500.0,1700.0,1929.0,1712.0,1660.0,2095.0,1726.0,1792.0,1716.0,1941.0,...,4000.0,3950.0,4400.0,4000.0,3500.0,3125.0,3125.0,3400.0,3600.0,3700.0


In [34]:
def attribute_query(attribute, year_type='Market_Year', countries=[]):
    df = usda[usda['Attribute_Description'] == attribute]
    if countries:
        df = df[df['Country_Name'].isin(countries)]
    return df.pivot(
        index='Country_Name', 
        columns=year_type, 
        values='Value'
    )

attribute_query('Arabica Production', countries=['Guatemala', 'Honduras', 'Nicaragua'])

Market_Year,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Country_Name,,,,,,,,,,,,,,,,,,,,,
Guatemala,1500.0,1700.0,1929.0,1712.0,1660.0,2095.0,1726.0,1792.0,1716.0,1941.0,...,4000.0,3950.0,4400.0,4000.0,3500.0,3125.0,3125.0,3400.0,3600.0,3700.0
Honduras,291.0,315.0,405.0,390.0,479.0,461.0,431.0,495.0,514.0,443.0,...,3550.0,3975.0,5600.0,4725.0,4400.0,5100.0,5300.0,7510.0,7600.0,7600.0
Nicaragua,437.0,432.0,535.0,533.0,610.0,513.0,421.0,556.0,504.0,563.0,...,1925.0,1740.0,2100.0,1925.0,2000.0,2100.0,2100.0,2600.0,2700.0,2500.0
